In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../..')

In [7]:
from src.wess_zumino_model import WessZuminoModel
from src.matrix_to_ps import matrix_to_pse
from src.binary_encodings import standard_encode
from src.qiskit_utilities import pauli_dict_to_op, op_to_trotter

from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms.eigensolvers import VQD
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import RealAmplitudes
from qiskit import BasicAer
from qiskit import qpy
from qiskit import QuantumCircuit, transpile

import scipy.sparse.linalg
import sympy as sp
import numpy as np

from ipywidgets import IntProgress
from IPython.display import display, clear_output 

import matplotlib.pyplot as plt
import matplotlib.lines as mlines

import json
import math
import os
import time

In [3]:
def potential(self, n):
    # m*q with m=1
    return -self.mass*self.qs[n]

N=2
wz=WessZuminoModel(N,1.0,potential,'dirichlet')

## TODO 

* Compare in system with and without SSYM preserved

In [16]:
cutoff=16
runtime = -time.time()
wz.construct_ham_matrix(cutoff)
runtime += time.time()
print("%0.1f seconds to construct hamiltonian" % runtime)

#e0=scipy.sparse.linalg.eigs(wz.hamMat,k=1,sigma=0.0)[0]

runtime = -time.time()
ps=matrix_to_pse(wz.hamMat, standard_encode)
runtime += time.time()
print("%0.1f seconds to convert to Pauli string" % runtime)

runtime = -time.time()
op=pauli_dict_to_op(ps.to_dict())
runtime += time.time()
print("%0.1f seconds to convert to operator" % runtime)

nq=math.floor(math.log2(wz.hamMat.shape[0]))
if not math.log2(wz.hamMat.shape[0]).is_integer():
    nq+=1
print("nq = %d" % nq)

1.2 seconds to construct hamiltonian
9.2 seconds to convert to Pauli string
0.3 seconds to convert to operator
nq = 10


In [17]:
from qiskit.primitives import Estimator, Sampler
from qiskit.algorithms.state_fidelities import ComputeUncompute
estimator = Estimator()
sampler=Sampler()
fidelity=ComputeUncompute(sampler)

ansatz = RealAmplitudes(nq, entanglement='circular', reps=2)

In [18]:
k=3
maxiter=10000
betas=[2,2,2]

counts=[]
values=[]
steps=[]

def callback(eval_count, params, value, meta, step):
    counts.append(eval_count)
    values.append(value)
    steps.append(step)

In [ ]:
for target in [1e-4, 1e-6, 1e-8]:
    counts=[]
    values=[]
    steps=[]
    runtime = -time.time()
    vqd=VQD(estimator, fidelity, ansatz, optimizer=COBYLA(maxiter=maxiter, tol=target), k=k, betas=betas, callback=callback)
    result=vqd.compute_eigenvalues(operator=op)
    runtime += time.time()
    print("Runtime: %0.1f seconds for target %e" % (runtime, target))

    counts=np.asarray(counts)
    steps=np.asarray(steps)
    values=np.asarray(values)
    for i in range(1,k+1):
        _counts=counts[np.where(steps==i)]
        _values=values[np.where(steps==i)]
        plt.plot(_counts,_values,label=r'$E_{{{}}}$'.format(i))
        if _values[-1] >= maxiter - 1:
            print("NOT CONVERGED")
        print("E%d=%.4g after %d iterations" % (i, _values[-1], _counts[-1]))
    plt.xlabel(r'Iteration')
    plt.ylabel(r'$E$')
    plt.yscale('log')
    plt.legend(loc='upper right')
    plt.show()